In [1]:
import numpy as np
import pandas as pd
import os

PATH = "/kaggle/input/tsp-cv"
PATH_TRAIN = os.path.join(PATH, "train.csv")
PATH_TEST = os.path.join(PATH, "test.csv")

In [2]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() \
      else "NOT AVAILABLE")

Tensor Flow Version: 2.6.2
Keras Version: 2.6.0

Python 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
Pandas 1.3.5
Scikit-Learn 1.0.1
GPU is available


2022-04-14 02:15:41.155488: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-14 02:15:41.211513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:15:41.292005: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:15:41.292751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [3]:
df_train = pd.read_csv(PATH_TRAIN)
df_test = pd.read_csv(PATH_TEST)

df_train['filename'] = df_train.id.astype(str) + ".jpg"
df_test['filename'] = df_test.id.astype(str) + ".jpg"

from sklearn.model_selection import train_test_split
df_train_cut, df_validate_cut = train_test_split(df_train, test_size=0.1, random_state = 42)
print(f"Training size: {len(df_train_cut)}")
print(f"Validate size: {len(df_validate_cut)}")

Training size: 14416
Validate size: 1602


In [4]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

WIDTH = 256
HEIGHT = 256

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=32, # Keeping the training batch size small USUALLY increases performance
        class_mode='raw')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=256, # Make the validation batch size as large as you have memory for
        class_mode='raw')

Found 14416 validated image filenames.
Found 1602 validated image filenames.


In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input

input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

base_model = ResNet50(
    include_top=False, weights=None, input_tensor=input_tensor,
    input_shape=None)

#base_model.summary()

2022-04-14 02:15:48.365814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:15:48.366691: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:15:48.367339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:15:48.368217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:15:48.368876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [6]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
model=Model(inputs=base_model.input,outputs=Dense(1)(x))

In [7]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError

# Important, calculate a valid step size for the validation dataset
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=[RootMeanSquaredError(name="rmse")])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, verbose=1, mode='auto',
        restore_best_weights=True)

history = model.fit(train_generator, epochs=200, steps_per_epoch=250, 
                    validation_data = val_generator, callbacks=[monitor],
                    verbose = 1, validation_steps=STEP_SIZE_VALID)



2022-04-14 02:15:50.111126: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200


2022-04-14 02:15:56.185645: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


250/250 [==============================] - 150s 554ms/step - loss: 162158112.0000 - rmse: 12734.1318 - val_loss: 700085184.0000 - val_rmse: 26459.1230
Epoch 2/200
250/250 [==============================] - 109s 433ms/step - loss: 73079552.0000 - rmse: 8548.6582 - val_loss: 53703643136.0000 - val_rmse: 231740.4688
Epoch 3/200
250/250 [==============================] - 104s 413ms/step - loss: 55945540.0000 - rmse: 7479.6753 - val_loss: 587316288.0000 - val_rmse: 24234.6094
Epoch 4/200
250/250 [==============================] - 99s 395ms/step - loss: 44720768.0000 - rmse: 6687.3589 - val_loss: 332640128.0000 - val_rmse: 18238.4238
Epoch 5/200
250/250 [==============================] - 96s 384ms/step - loss: 33955992.0000 - rmse: 5827.1772 - val_loss: 158689264.0000 - val_rmse: 12597.1924
Epoch 6/200
250/250 [==============================] - 96s 385ms/step - loss: 30119182.0000 - rmse: 5488.0947 - val_loss: 6613725184.0000 - val_rmse: 81324.8125
Epoch 7/200
250/250 [======================

In [8]:
submit_datagen = ImageDataGenerator(rescale = 1./255)

submit_generator = submit_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=PATH,
        x_col="filename",
        batch_size = 1,
        shuffle = False,
        target_size=(HEIGHT, WIDTH),
        class_mode=None)

submit_generator.reset()
pred = model.predict(submit_generator,steps=len(df_test))
df_submit = pd.DataFrame({"id":df_test['id'],'distance':pred[:,0].flatten()})
df_submit.to_csv("submission.csv",index = False)
df_submit

Found 4005 validated image filenames.


,id,distance
0,16019,7646.633301
1,16020,57419.515625
2,16021,11251.871094
3,16022,6244.526367
4,16023,14903.236328
...,...,...
4000,20019,7500.638672
4001,20020,9795.900391
4002,20021,9906.341797
4003,20022,15410.347656
